# Chap 6. 데이터 로딩, 저장, 파일형식

o 세가지 입출력 종류: 텍스트파일, 데이터베이스, 웹API

In [12]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [39]:
%pwd

'C:\\Dropbox (개인용)\\2018KPUWork\\10 강의-201801학기\\201803 빅데이터\\03 강의교안\\파이썬데이터분석'

## 6.1 텍스트 파일포맷으로 읽고 쓰기 
o padas 제공 파일 파싱하여 DataFrame으로 읽어오는 함수
 - read_csv : 
   - input파일, URL 또는 파일과 유사항 객체
   - 데이터 구분자 쉼표(,)를 기본으로 데이터 읽기
 - read_table : 
   - input파일, URL 또는 파일과 유사항 객체
   - 데이터 구분자 탭(\t)를 기본으로 데이터 읽기
 - read_fwf : 
   - input파일, URL 또는 파일과 유사항 객체
   - 고정폭 컬럼형식에서 데이터 읽기
 - read_clipboard : 
   - 클립보드에 있는 데이터를 읽어오는 read_table 함수, 웹페이지에서 표를 긁어올 때 유용

o pandas 파일 파싱 함수 옵션  
 - 색인: 반환하는 DataFrame에서 하나 이상의 칼럼을 색인으로 지정할 수 있다. 파일이나 사용자로부터 칼럼의 이름을 받거나 아무것도 받지 않을 수 있다.
 - 자료형 추론과 데이터 변환: 사용자 정의 값 변환과 비어있는 값을 위한 사용자 리스트를 포함한다.
     ==> 자료형 추론: 값을 보고 적절한 자료형 추정 (숫자, 문자, 불리언 등) 
 - 날짜 분석: 여러 칼럼에 걸쳐 있는 날짜와 시간 정보를 하나의 칼럼에 조합해서 결과에 반영한다.
 - 반복: 여러 파일에 걸쳐 있는 자료를 반복적으로 읽어올 수 있다.
 - 정제되지 않는 데이터 처리: 로우나 꼬리말, 주석 건너뛰기 또는 천 단위마다 쉼표로 구분된 숫자 같은 사소한 일을 처리해준다.
 

In [40]:
#!cat ch06/ex1.csv  # cat은 리눅스 명령어임, 
                    # 윈도우 명령으로 대체, 파일 패스기준: 위의 !pwd 파일을 기준으로 지정필요./ -> \ 변환필요
!type ch06\ex1.csv  

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [41]:
df = pd.read_csv('ch06/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [82]:
pd.read_table('ch06/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [83]:
!type ch06\ex2.csv
# 주의: header가 없는 csv 파일

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [86]:
# 헤더 미지정으로 읽기
pd.read_csv('ch06/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [87]:
# 헤더 지정해서 읽기
pd.read_csv('ch06/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [88]:
# header 지정하고, 특정 컬럼을 row index로 활용 
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('ch06/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [89]:
# 계층적 색인: index_col에서 컬럼헤더를 두개 이상 지정
!type ch06\csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [90]:
parsed = pd.read_csv('ch06/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

o 고정된 구분자가 없다면 read_table의 구분자로 정규표현식(파이썬 – 정규식표현식(Regular Expression))을 사용하면 된다.

In [95]:
list(open('ch06/ex3.txt'))  # 특수문자까지 표시 함. 
#!type ch06\ex3.txt -
# 주의: 첫째 연의 첫 컬럼이 없음 --> 첫 컬럼은 로우 인덱스로 자동 추론함.

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [96]:
result = pd.read_table('ch06/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [98]:
# skiprows 옵션 
!type ch06\ex4.csv
pd.read_csv('ch06/ex4.csv', skiprows=[0, 2, 3])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


o (비교) read_csv vs. read_table
 - 둘 다 사용을 해도 되지만 
 - read_csv는 csv 파일에만 
 - 나머지 특별한 경우를 read_table로 활용하는 것을 권장 

o 공백표시 자동처리
 - 공백표시: NA, -1, #IND, NULL  (참고: #IND : idefinite, the value when divided by 0 
 - 자동인식하여 NaN 과 같은 NULL 밸류로 인식

In [99]:
!type ch06\ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [101]:
result = pd.read_csv('ch06/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [102]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [103]:
# na_values 옵션: Null(NaN)으로 해석할 리스트나 문자열을 받아, 그 값들을 Null (NaN) 으로 대체한다. 
# result = pd.read_csv('ch06/ex5.csv', na_values=['NULL'])
result = pd.read_csv('ch06/ex5.csv', na_values=['foo'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


o 열마다 다른 값들을 Null로 처리하도록 지정 가능 (key: na_values list)의 사전으로 지정 

In [104]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('ch06/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


o read_csv/read_table 함수 인자
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html 

 - path	파일 시스템에서의 위치, URL, 파일 객체를 나타내는 문자열
 - sep or delimiter	필드를 구분하기 위해 사용할 연속된 문자나 정규표현식
 - header	칼럼의 이름으로 사용할 로우의 번호, 기본 값은 0(첫 로우)이며 헤더가 없으면 None으로 지정할 수 있다.
 - index_col	색인으로 사용할 칼럼 번호나 이름, 계층적 색인을 지정할 경우 리스트를 넘길 수 있다.
 - names	컬럼 이름으로 사용할 리스트. header = None과 함께 사용한다.
 - skiprows	파일의 시작부터 무시할 로우의 개수 또는 무시할 로우 번호가 담긴 리스트
 - na_values	NA 값으로 처리할 값들의 나열
 - comment	주석으로 분류되어 파싱하지 않을 문자 혹은 문자열
 - parse_dates	날짜를 datetime으로 변환할지의 여부. 기본값은 False이며, True일 경우 모든 칼럼에 다 적용된다. 리스트를 넘기면 변환할 칼럼을 지정할 수 있는데, [1, 2, 3]을 넘기면 각각의 칼럼을 datetime으로 변환하고, [[1, 3]]을 넘기면 1, 3번 칼럼을 조합해서 하나의 datetime으로 변환한다.
 - keep_date_col	여러 칼럼을 datetime으로 변환했을 경우 원래 칼럼을 남겨둘지의 여부. 기본값은 False
 - converters	변환 시 칼럼에 적용할 함수를 지정한다. 예를 들어 {'foo': f}는 'foo'칼럼에 f 함수를 적용한다. 전달하는 사전의 키 값은 칼럼 이름이나 번호가 될 수 있다.
 - dayfirst	모호한 날짜 형식일 경우 국제 형식으로 간주한다(7/6/2012는 2012년 6월 7일로 간주한다). 기본값은 False
 - date_parser	날짜 변환 시 사용할 함수
 - nrows	파일의 첫 일부만 읽어올 때 처음 몇 줄을 읽을 것인지 지정한다.
 - iterator	파일을 조금씩 읽을 때 사용하도록 TextParser 객체를 반환하도록 한다. 기본값은 False
 - chunksize	TextParser 객체에서 사용할, 한 번에 읽을 파일의 크기
 - skip_footer	무시할 파일의 마지막 줄 수
 - verbose	파싱 결과에 대한 정보를 출력한다. 숫자가 아닌 값들이 들어있는 칼럼이면서 누락된 값이 있다면 줄 번호를 출력한다. 기본값은 False
 - encoding	유니코드 인코딩 종류를 지정한다. UTF-8로 인코딩된 텍스트일 경우 'utf-8'로 지정한다.
 - squeeze	로우가 하나뿐이라면 Series 객체를 반환한다. 기본값은 False
 - thousands	숫자를 천 단위로 끊을 때 사용할 ', '나 '.' 같은 구분자

### 6.1.1 텍스트 파일 조금씩 읽어오기 (Reading text files in pieces)

In [105]:
result = pd.read_csv('ch06/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


In [106]:
# 상위 nrows만 읽어오기
pd.read_csv('ch06/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [107]:
# chunk 단위로 조각내어 반복하여 읽어오기
chunker = pd.read_csv('ch06/ex6.csv', chunksize=1000)
chunker

In [108]:
chunker = pd.read_csv('ch06/ex6.csv', chunksize=1000)

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [110]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### 6.1.2 텍스트포맷으로 데이터 쓰기 (Writing data out to text format)

In [111]:
data = pd.read_csv('ch06/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [112]:
data.to_csv('ch06/out.csv')
!type ch06\out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [113]:
# 표준출력으로 쓰기
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [114]:
# NULL으로 쓸 문자 지정하기 
# 기본적으로 헤더와 인덱스를 기록함. 
# data.to_csv(sys.stdout, na_rep='NULL')
data.to_csv(sys.stdout, na_rep='NALL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NALL
1,two,5,6,NALL,8,world
2,three,9,10,11.0,12,foo


In [115]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [116]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [117]:
dates = pd.date_range('1/1/2000', periods=7)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

o Series에도 to_csv method 존재

In [1]:
ts = Series(np.arange(7), index=dates)
ts.to_csv('ch06/tseries.csv')
!type ch06\tseries.csv

NameError: name 'Series' is not defined

In [32]:
Series.from_csv('ch06/tseries.csv', parse_dates=True)
# pd.read_csv('ch06/tseries.csv', index_col=0, header=None) 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:2890: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

### 6.1.3 수동으로 구분형식 처리 (Manually working with delimited formats) 

In [54]:
!type ch06\ex7.csv
% pwd

"a","b","c"
"1","2","3"
"1","2","3","4"


'C:\\Dropbox (개인용)\\2018KPUWork\\10 강의-201801학기\\201803 빅데이터\\03 강의교안\\파이썬데이터분석'

o 구분자가 한글자인 csv 파일 처리
 - 오픈된 파일을 csv.reader에 전달하여 for 문을 처리 

In [55]:
import csv
f = open('ch06/ex7.csv')

reader = csv.reader(f)

In [56]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [57]:
lines = list(csv.reader(open('ch06/ex7.csv')))
header, values = lines[0], lines[1:]
header
values

[['1', '2', '3'], ['1', '2', '3', '4']]

In [58]:
data_dict = {h: v for h, v in zip(header, zip(*values))} 
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [6]:
a = [1,2,3]
b = [4,5,6]
z = list(zip(a,b))
print(z)
print(zip(*z))

[(1, 4), (2, 5), (3, 6)]
[(1, 2, 3), (4, 5, 6)]


#### o csv.Dialect를 상속: 다양한 파일형태를 처리하기 위해 조정
 - 다양한 구분자
 - 문자열을 둘러싸는 방법
 - 개행문자 

In [59]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
    
reader = csv.reader(f, dialect=my_dialect)

#### CSV Note
- 좀 더 복잡하거나 구분자가 한 글자를 초과하는 고정 길이를 가진다면 csv 모듈을 사용할 수 없다.
- 이런 경우에는 줄을 나누고 문자열의 split 메서드나 정규표현식 메서드인 re.split 등을 이용해서 가공하는 작업을 해야 한다

#### CSV 파일 기록

In [60]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)   # quoting=csv.QUOTE_NONE
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [61]:
!type mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9


### JSON data
- JSON(JavaScript Object Notation)은 웹브라우저와 다른 애플리케이션이 HTTP 요청으로 데이터를 보낼 때 널리 사용하는 표준 파일 형식 중 하나다.
- JSON은 CSV 같은 표 형식의 텍스트보다 좀 더 유연한 데이터 형식이며, JSON 데이터의 예는 다음과 같다.

In [5]:
# json은 python에서처럼 '으로 하면 안된다. 
# 현재 """로 감싸 문자열로 저장되어 있기 때문에 
# javascript에서는 '를 string 값으로 인식하지 않아서 에러 발생
# ==> key, value를 모두 doluble quote(") 를  사용하여 감쌈
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

In [65]:
# python 형태로 변환: json.load( ) 
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 25, 'name': 'Scott', 'pet': 'Zuko'},
  {'age': 33, 'name': 'Katie', 'pet': 'Cisco'}]}

In [66]:
# python 객체를 json형태로변환 
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"}, {"name": "Katie", "age": 33, "pet": "Cisco"}]}'

In [67]:
# DataFrame 만들기: json 객체의 리스트를 생성자로 넘김. 
siblings = DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,25
1,Katie,33


In [68]:
siblings.to_json(orient='split')

'{"columns":["name","age"],"index":[0,1],"data":[["Scott",25],["Katie",33]]}'

In [70]:
siblings.to_json(orient='index')

'{"0":{"name":"Scott","age":25},"1":{"name":"Katie","age":33}}'

In [71]:
siblings.to_json(orient='records')

'[{"name":"Scott","age":25},{"name":"Katie","age":33}]'

In [72]:
siblings.to_json(orient='table')

'{"schema": {"fields":[{"name":"index","type":"integer"},{"name":"name","type":"string"},{"name":"age","type":"integer"}],"primaryKey":["index"],"pandas_version":"0.20.0"}, "data": [{"index":0,"name":"Scott","age":25},{"index":1,"name":"Katie","age":33}]}'

### 6.1.5 XML and HTML, Web scraping

#### lxml
- 아주 큰 파일을 빠르게 처리 가능
- 여러 종류의 인터페이스 제공
- lxml.html, lxml.objectify를 이용해서 HTML, XML 처리

#### HTML
- 대부분의 웹사이트는 딱 필요한 내용만 들어있는 JSON이나 XML을 많이 사용하지 않고 HTML을 사용

In [19]:
# urllib2 is named as urllib.request in python 3. 
from lxml.html import parse
#from urllib2 import urlopen
from urllib.request import urlopen

# See https://finance.yahoo.com/quote/AAPL/options?ltr=1
# right mouse click button --> 소스보기: <a> 링크 찾아 attribute href 보기 
parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))


doc = parsed.getroot()

In [20]:
parsed
# doc

In [21]:
links = doc.findall('.//a')  # XPath형태: 현재 이하의 모든 곳의 a element를 찾아 리스트로 반환  
links[15:20]

[<Element a at 0x141d3481d18>,
 <Element a at 0x141d3481d68>,
 <Element a at 0x141d3481db8>,
 <Element a at 0x141d3481e08>,
 <Element a at 0x141d3481e58>]

In [76]:
lnk = links[28]
lnk

<Element a at 0x141d34a7188>

In [77]:
# lnk.get('href')   # Element.get() : 주어진 key값의 값을 리턴 
lnk.text_content() # Returns the text content of the element and its children, with no markup.
                   # 소스보기에서 AAPL180504C00148000 찾아보기 바람.

'AAPL180504C00148000'

In [78]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-20:] # 소스보기에서 아래 값들을 찾아보기 바람.
# urls[20:30]

['/quote/AAPL/options?strike=false&straddle=false',
 'https://help.yahoo.com/kb/index?page=content&y=PROD_FIN_DESK&locale=en_US&id=SLN2310',
 'https://help.yahoo.com/kb/index?page=content&y=PROD_FIN_DESK&locale=en_US',
 'https://yahoo.uservoice.com/forums/382977',
 'http://info.yahoo.com/privacy/us/yahoo/',
 'http://info.yahoo.com/relevantads/',
 'http://info.yahoo.com/legal/us/yahoo/utos/utos-173.html',
 'https://twitter.com/YahooFinance',
 'https://facebook.com/yahoofinance',
 'http://yahoofinance.tumblr.com',
 '/',
 '/watchlists',
 '/portfolios?bypass=true',
 '/screener',
 '/calendar',
 '/industries',
 '/personal-finance',
 '/tech',
 '/topic/yahoo-finance-podcast',
 '/live/facebook']

In [79]:
tables = doc.findall('.//table')
tables
calls = tables[0]
puts = tables[1]

In [80]:
rows = calls.findall('.//tr')

In [81]:
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content() for val in elts]

In [83]:
_unpack(rows[0], kind='th')

['Contract Name',
 'Last Trade Date',
 'Strike',
 'Last Price',
 'Bid',
 'Ask',
 'Change',
 '% Change',
 'Volume',
 'Open Interest',
 'Implied Volatility']

In [84]:
_unpack(rows[1], kind='td')

['AAPL180504C00130000',
 '2018-04-24 10:37AM EDT',
 '130.00',
 '34.20',
 '31.95',
 '32.75',
 '0.00',
 '-',
 '3',
 '11',
 '77.34%']

In [85]:
from pandas.io.parsers import TextParser

def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data, names=header).get_chunk()

In [87]:
call_data = parse_options_data(calls)
call_data[:10]

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,AAPL180504C00130000,2018-04-24 10:37AM EDT,130.0,34.20,31.95,32.75,0.00,-,3,11,77.34%
1,AAPL180504C00135000,2018-04-26 9:32AM EDT,135.0,29.14,26.90,27.75,0.00,-,1,39,53.13%
2,AAPL180504C00140000,2018-04-27 3:51PM EDT,140.0,22.85,22.30,22.60,-1.34,-5.54%,44,15,66.60%
3,AAPL180504C00145000,2018-04-25 3:42PM EDT,145.0,19.18,17.65,18.05,0.00,-,10,162,71.58%
4,AAPL180504C00146000,2018-04-04 4:39PM EDT,146.0,24.92,23.30,24.15,0.00,-,5,0,196.31%
5,AAPL180504C00147000,2018-04-27 12:11PM EDT,147.0,15.35,15.70,16.25,-2.49,-13.96%,1,55,68.85%
6,AAPL180504C00148000,2018-04-27 9:37AM EDT,148.0,15.93,14.75,15.30,0.48,+3.11%,2,2,66.75%
7,AAPL180504C00149000,2018-04-24 3:55PM EDT,149.0,14.65,13.55,14.15,0.00,-,56,110,58.25%
8,AAPL180504C00150000,2018-04-27 3:58PM EDT,150.0,13.00,12.85,13.00,-1.45,-10.03%,"3,275",428,57.03%
9,AAPL180504C00152500,2018-04-27 3:59PM EDT,152.5,10.87,10.65,10.85,-2.21,-16.90%,438,53,55.76%


In [88]:
put_data = parse_options_data(puts)
put_data[:10]

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,AAPL180504P00120000,2018-04-27 12:59PM EDT,120.0,0.02,0.00,0.02,0.00,-,31,31,90.63%
1,AAPL180504P00130000,2018-04-27 3:49PM EDT,130.0,0.01,0.00,0.02,-0.01,-50.00%,29,535,68.75%
2,AAPL180504P00135000,2018-04-27 2:47PM EDT,135.0,0.03,0.00,0.04,-0.04,-57.14%,106,353,62.50%
3,AAPL180504P00140000,2018-04-27 3:49PM EDT,140.0,0.04,0.04,0.06,-0.02,-33.33%,644,695,57.42%
4,AAPL180504P00141000,2018-04-27 1:14PM EDT,141.0,0.10,0.08,0.10,0.00,-,137,0,60.16%
5,AAPL180504P00142000,2018-04-27 3:59PM EDT,142.0,0.09,0.08,0.10,0.01,+12.50%,119,79,57.62%
6,AAPL180504P00143000,2018-04-27 3:57PM EDT,143.0,0.11,0.09,0.12,0.00,-,405,216,56.35%
7,AAPL180504P00144000,2018-04-27 3:02PM EDT,144.0,0.12,0.12,0.15,-0.02,-14.29%,"1,396",302,56.06%
8,AAPL180504P00145000,2018-04-27 3:53PM EDT,145.0,0.14,0.16,0.18,-0.04,-22.22%,736,"1,012",55.76%
9,AAPL180504P00146000,2018-04-27 3:17PM EDT,146.0,0.18,0.20,0.22,0.01,+5.88%,81,295,55.18%


##### o 옵션에 대하여 
 - https://ko.wikipedia.org/wiki/%EC%98%B5%EC%85%98_(%EA%B8%88%EC%9C%B5)
 - 옵션(option)은 파생 상품의 일종이며, 미리 결정된 기간 안에 특정 상품을 정해진 가격으로 사고 팔 수 있는 권리
 - call option: 특정 금융 상품을 정해진 가격에 매입할 수 있는 권리를 가진 매입 옵션(call option)
 - put option: 매도할 수 있는 권리를 가진 매도 옵션(put option)으로 나뉜다.

#### Parsing XML with lxml.objectify

In [1]:
%cd ch06/mta_perf

E:\Dropbox (개인용)\2018KPUWork\10 강의-201801학기\201803 빅데이터\03 강의교안\파이썬데이터분석\ch06\mta_perf


In [3]:
!type Performance_MNR.xml  # chome에서  xml  파일 보기

<?xml  version="1.0" encoding="ISO-8859-1"?>
<PERFORMANCE>
<INDICATOR>
  <INDICATOR_SEQ>28445</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>On-Time Performance (West of Hudson)</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.
</DESCRIPTION>
  <PERIOD_YEAR>2008</PERIOD_YEAR>
  <PERIOD_MONTH>1</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>95.00</YTD_TARGET>
  <YTD_ACTUAL>96.90</YTD_ACTUAL>
  <MONTHLY_TARGET>95.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>96.90</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICAT

  <PERIOD_MONTH>2</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL>97.10</YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>95.20</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28461</INDICATOR_SEQ>
  <PARENT_SEQ>28445</PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Pascack Valley Line - OTP</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. Metro-North Railroad contracts with New Jersey Transit to operate service on the Pascack Valley Line.</DESCRIPTION>
  <PERIOD_YEAR>2010</PERIOD_YEAR>
  <PERIOD_MONTH>3</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATO

  <YTD_TARGET>3.20</YTD_TARGET>
  <YTD_ACTUAL>2.27</YTD_ACTUAL>
  <MONTHLY_TARGET>3.20</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>2.27</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28530</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Customer Injury Rate</INDICATOR_NAME>
  <DESCRIPTION>Any injury to a customer as a result of an incident within/on railroad property. Included are injuries that result from an assault or crime. The rate is injuries per million customers.</DESCRIPTION>
  <PERIOD_YEAR>2009</PERIOD_YEAR>
  <PERIOD_MONTH>2</PERIOD_MONTH>
  <CATEGORY>Safety Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>D</DESIRED_CHANGE>
  <INDICATOR_UNIT>-</INDICATOR_UNIT>
  <DECIMAL_PLACES>2</DECIMAL_PLACES>
  <YTD_TARGET>3.20</YTD_TARGET>
  <YTD_ACTUAL>2.10</YTD_ACTUAL>
  <MONTHLY_TARGET>3.20</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>1.92</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28530</INDIC

<INDICATOR>
  <INDICATOR_SEQ>28627</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Employee Lost Time and Restricted Duty Rate</INDICATOR_NAME>
  <DESCRIPTION>An employee lost time injury or illness is one that prevents an employee from returning to work for at least one full shift. The rate is injuries and illnesses per 200,000 worker hours.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>1</PERIOD_MONTH>
  <CATEGORY>Safety Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>D</DESIRED_CHANGE>
  <INDICATOR_UNIT>-</INDICATOR_UNIT>
  <DECIMAL_PLACES>2</DECIMAL_PLACES>
  <YTD_TARGET>1.70</YTD_TARGET>
  <YTD_ACTUAL>3.89</YTD_ACTUAL>
  <MONTHLY_TARGET>1.70</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>3.89</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28627</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Employee Lost Time and Rest

  <PERIOD_YEAR>2008</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>98.00</YTD_TARGET>
  <YTD_ACTUAL>97.60</YTD_ACTUAL>
  <MONTHLY_TARGET>98.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>97.20</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28345</INDICATOR_SEQ>
  <PARENT_SEQ>55526</PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Hudson Line - OTP</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time.</DESCRIPTION>
  <PERIOD_YEAR>2009</PERIOD_YEAR>
  <PERIOD_MONTH>1</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGE

  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>98.30</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>98.30</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28346</INDICATOR_SEQ>
  <PARENT_SEQ>55526</PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Harlem Line - OTP</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>98.30</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>98.30</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>28347</INDICATO

</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>55526</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>On-Time Performance (East of Hudson)</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. East of Hudson service includes the Harlem, Hudson and New Haven lines.</DESCRIPTION>
  <PERIOD_YEAR>2010</PERIOD_YEAR>
  <PERIOD_MONTH>11</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.70</YTD_TARGET>
  <YTD_ACTUAL>97.70</YTD_ACTUAL>
  <MONTHLY_TARGET>97.70</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>96.60</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>55526</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>On-Time Perfo

  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational systemwide. The availability rate is based on physical observations performed the morning of regular business days only. This is a new indicator the agency began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2009</PERIOD_YEAR>
  <PERIOD_MONTH>10</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET></YTD_TARGET>
  <YTD_ACTUAL>96.00</YTD_ACTUAL>
  <MONTHLY_TARGET></MONTHLY_TARGET>
  <MONTHLY_ACTUAL>98.70</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational systemwide. The availabili


Performance_MNR.xml


지정된 파일을 찾을 수 없습니다.
다음 내용 진행 중 오류 발생: #.
지정된 파일을 찾을 수 없습니다.
다음 내용 진행 중 오류 발생: chome에서.
지정된 파일을 찾을 수 없습니다.
다음 내용 진행 중 오류 발생: xml.
지정된 파일을 찾을 수 없습니다.
다음 내용 진행 중 오류 발생: 파일.
지정된 파일을 찾을 수 없습니다.
다음 내용 진행 중 오류 발생: 보기.


In [4]:
from lxml import objectify

path = 'Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [8]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']


# 참고) xml.ElementTree https://docs.python.org/2/library/xml.etree.elementtree.html#element-objects

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():   
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval # element.pyval attribute that returns the value as plain Python type
    data.append(el_data)

In [13]:
data

[{'AGENCY_NAME': 'Metro-North Railroad',
  'CATEGORY': 'Service Indicators',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\n',
  'FREQUENCY': 'M',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'INDICATOR_UNIT': '%',
  'MONTHLY_ACTUAL': 96.9,
  'MONTHLY_TARGET': 95.0,
  'PERIOD_MONTH': 1,
  'PERIOD_YEAR': 2008,
  'YTD_ACTUAL': 96.9,
  'YTD_TARGET': 95.0},
 {'AGENCY_NAME': 'Metro-North Railroad',
  'CATEGORY': 'Service Indicators',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\

In [14]:
perf = DataFrame(data) #사전의 리스트를 이용한 DataFrame 생성
perf

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95
5,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,94.4,95,6,2008,96.2,95
6,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96,95,7,2008,96.2,95
7,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,8,2008,96.2,95
8,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,93.7,95,9,2008,95.9,95
9,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,10,2008,96,95


## 6.2 Binary data formats

In [15]:
cd ../..

E:\Dropbox (개인용)\2018KPUWork\10 강의-201801학기\201803 빅데이터\03 강의교안\파이썬데이터분석


In [34]:
frame = pd.read_csv('ch06/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [35]:
frame.to_pickle('ch06/frame_pickle')

In [36]:
pd.read_pickle('ch06/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 6.2.1 Using HDF5 format
https://support.hdfgroup.org/HDF5/doc/H5.intro.html#Intro-WhatIs

https://kr.mathworks.com/help/matlab/import_export/importing-hierarchical-data-format-hdf5-files.html

o 디스크에 이진 형식으로 저장된 대용량의 과학 자료를 효율적으로 읽고 쓸 수 있는 다양한 도구 존재

o 산업 기준에 맞는 인기 라이브러리중 하나가 HDF5(Hierarchical Data Format), 

o 계층적 데이터 형식

o 내부적으로 파일 시스템 같은 노드 구조

o 여러 개의 데이터셋을 저장하고 부가 정보 기록 가능

o 다양한 압축 기술을 사용해서 on-the-fly(실시간) 압축 지원

o 반복되는 패턴을 가진 데이터 좀 더 효과적 저장

o 메모리에 모두 적재할 수 없는 엄청나게 큰 데이터를 아주 큰 배열에서 필요한 만큼의 작은 부분들만 읽기 가능

o 두가지 HDF5 패키지 
- PyTables: HDF5를 추상화하여 여러가지 유연한 데이터 컨테이너와 테이블 색인, 질의 기능 그리고 외부 메모리 연산(out-of-core, external memory algorithm) 지원
- h5py: 직접적이지만 고수준의 HDF5 API에 대한 인터페이스 제공
pandas는 PyTable를 이용한 HDFStore라는 가벼운 사전 클래스를 통해 pandas 객체를 저장

In [37]:
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [38]:
store['obj1']

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [39]:
store['obj1_col']

0    1
1    5
2    9
Name: a, dtype: int64

In [40]:
store.close()
os.remove('mydata.h5')

#### 6.2.2 마이크로소프트 엑셀 파일에서 데이터 읽어오기
o pandas는 ExcelFile 클래스를 통해 마이크로소프트 엑셀 2003 이후 버전의 데이터를 읽기 가능

o 내부적으로 ExcelFile 클래스는 xlrd, openpyxl 패키지 활용. 사용하기 전에 먼저 설치
 - anaconda prompt >>> conda install xlrd
  

In [41]:
# %pwd
xls_file = pd.ExcelFile('data.xlsx')

In [42]:
# DataFrame 생성
table = xls_file.parse('Sheet1')
table

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo
3,20,21,24,232,Kim Jeong Eun


### 6.3 Interacting with HTML and Web APIs 
o JSON  형식으로 데이터를 제공하는 공개 API 를 제공하는 웹사이트 활용 방법

In [49]:
import requests
# url = 'https://api.github.com/repos/pydata/pandas/milestones/28/labels'
url = 'http://search.twitter.com/search.json?q=python%20pandas'
resp = requests.get(url)
resp

<Response [410]>

In [51]:
import json
data = json.loads(resp.text)

data[:5]

TypeError: unhashable type: 'slice'

In [46]:
issue_labels = DataFrame(data, columns=['AGENCY_NAME','CATEGORY'])
issue_labels

AssertionError: 2 columns passed, passed data had 4 columns

## 6.4 Interacting with databases
o 대부분의 애플리케이션은 텍스트 파일에서 데이터를 읽어오지 않음
 - 왜냐하면 대용량의 데이터를 저장하기에 텍스트 파일은 상당히 비효율적
 
o SQL 기반의 관계형 데이터 베이스가 많이 사용됨. : MySQL, Oracle, SQLite3

o NoSQL: 비정형 대용량 데이터베이스

o SQL에서 데이터를 읽어와서 DataFrame에 저장하는 방법은 꽤 직관적

o SQLite3을 이용한 실습

In [52]:
# sqlite3 패키지 설명: https://docs.python.org/2/library/sqlite3.html

import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:') # dbname을 넣어야 하나, :memory 의 경우 db를 메모리에 생성 
con.execute(query)
con.commit()

In [53]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [54]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [55]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [56]:
DataFrame(rows, columns=list(zip(*cursor.description))[0])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [30]:
import pandas.io.sql as sql
sql.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


### 6.4.1 MongoDB : skip 